In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
train_data = pd.read_csv("../input/train.csv")
#test_data = pd.read_csv("../input/test.csv")

In [ ]:
train_data.head()

# Exploratory Data Analysis

In [ ]:
train_data.describe()

In [ ]:
train_data["Sex"].value_counts()

Note that we have more males than females

In [ ]:
sns.countplot(x = "Sex", hue ="Survived",data = train_data, palette = "Blues");

In [ ]:
sns.countplot(x = "Pclass", hue ="Survived",data = train_data, palette = "Blues");

In [ ]:
sns.countplot(x = "Pclass", hue ="Sex",data = train_data, palette = "Blues");

In [ ]:
sns.barplot(x = "Embarked", y = "Survived", data = train_data)

In [ ]:
features = ["Pclass","Sex","Age","Fare","Parch","SibSp","Embarked"]
y = train_data["Survived"]
train_data = train_data[features]

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data['Age'].fillna(train_data['Age'].median(),inplace=True)

In [ ]:
train_data['Embarked'].fillna(train_data['Embarked'].value_counts().index[0], inplace=True)

In [ ]:
p = {1:'1st',2:'2nd',3:'3rd'} 
train_data['Pclass'] = train_data['Pclass'].map(p)

In [ ]:
categorical_df = train_data[['Pclass','Sex','Embarked']]
one_hot_encode = pd.get_dummies(categorical_df,drop_first=True) 
train_data = train_data.drop(['Pclass','Sex','Embarked'],axis=1)
train_data = pd.concat([train_data,one_hot_encode],axis=1)

In [ ]:
X = train_data
train_X, test_X, train_y, test_y = train_test_split(X,y,random_state = 0)

# Training Model

In [ ]:

clf = DecisionTreeClassifier()
clf.fit(train_X,train_y)

In [ ]:
pred = clf.predict(test_X)

In [ ]:
from sklearn import metrics
metrics.accuracy_score(test_y, pred)

In [ ]:
#my_submission = pd.DataFrame({'PassengerId': testingData.PassengerId, 'Survived': pred})

In [ ]:
#my_submission.to_csv('submission.csv', index=False)